In [1]:
#imports 
import string
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
with open('./amharic_all_text.txt', 'r', encoding="utf-8") as f:
    dirty_corp = f.read().encode().decode('unicode_escape')

dirty_corp = dirty_corp.split('^^^^^^')
print(f'Num of texts: {len(dirty_corp)}')

Num of texts: 1337


In [3]:
dirty_corp[1][1:1000]

'{"page_number": 1, "text": "ትምህርት 5\n\nየገባብ ክፍል:-  (3፳ ቆሮግቶስ 9;35-16)\n\nጥ2ቄ 3, በቁጥር 15 ላይ «እኀጺህ እግጺሆኀግልኻ ይህኻ አስጽፍምቅ»ቅ ሲል\nሞኀ ማለቱ ነው?\n\nጥናቄ 2. በቁጥር 35 ላይ ፋትምክሕት» የሚለው ምኑገ ነው?\n\nጥዩቄ 5, በቁጥር 37 መሠረት ሐዋርደው ወገኻጌልኀ ይሰብክ የነበረው\nበፈቃጽ ነበር ወይስ ፻ለፈቃጽሃ\n\nጥ5ናቄ 4, በቁጥር 38 ላይ ወኀግጌስኻ የመስበክ ጸመወዙ ምገኀጽነው ይላል?\n\nቁጥር 35;-- ክላይ ክዘረዘረቸው  መብቶች  በአጎገጹም እገኳ\n#ልተጠቀመ እግጸሆነ ይመሰክረሪለል።። - በዚህ ገዜ በአእምሮው ዋመጣበት\nምናልባት የቆሮኻቶስ ክርስቲፎኖች «ይህጎገ ሁሉ አሁኻ የሚናገረው ጸመወዝ\nፈልጐ ነው ይሉኝ ይሆናለ!፤» የሚለው ግምት ነበር:  ይህም  እኀጻለሆነ\nበክባጽ አነጋገር ዌሪጋግጥላቸዋል።- ማም ትምክህቴኻ ክኻቱ ከሚጸርግብኝ\nሞት ይሻለናልና=»\n\nትምክህቱ ወግጌለኘኻ በነዓ መስበኩ ነው። ግኻ አሁኻ ፃሳቡኘ ቀይሮ\nጸመጩዝ መቀበስ ቢጃምር ፳ ትምክህቱ ይቀረል።ሮ። ትምክህቱ የትሰቢት\nሳይሆገኻ ጸመወዝ  በመቀበለ ሊመጣ የላነበረውኀ- ፃወገግጌልገ እኀቅፋት\nበማስወገጹ ነበር።- በቆሮግቶስክ ቤተ ክርስቲፀ9ሓጣግ ውስጥ ሐዋርያውግኀ\nየማይወጹ ሐሰተኛና ጸመወጩዝ ወጻጽ አእስተማሪዎች ተነሥተው ነበርና\nየሐዋርፀው አቋም  ለእውነት በጣም አስፈላጊ ነበር።።-  እነዚህም ሐሰተኛ\nከስተማሪዎች እርሱሞ ጸመመዝግኀ ተቀብሎ ክእነርሱ እኩፅ በምሞሆገ\nእ፲፲፲ጻያሳጣቸው ይፈልጉ ነበር፲ (12፳ ቆሮ,33:32):\n\nጥ፻ደቄ 5, በአገጻግጽ ቤተ ክርስቲሄኻ ለጸመወዝ ብቻ ብለው የሚዩገለግሉ\nመሪዎች ከሉ"ኾ"  ይህ ዓይነት ፃሳብ እገጹት ለወግጌልና ለቤተ\nክርስቲ፻ግ እግቅፋት ሊሆገ

## Text Cleaning

In [4]:
class TextCleaner():
    """
    Amharic text cleaner
    """
    def __init__(self):
        self.min_ascii = '1200'
        self.max_ascii = '137F'
        self.punctuation = string.punctuation + '።'+'፤'+'፥'+'፦'+'፧'+'፨'+'«'+'»'+'-'+':'+'፣'+'፡፡'+'*'+''
        self.one_space_regex = r"\s((\s)(\s+)?)?"
        self.text = None

        self.dict_punct = dict(zip(list(self.punctuation), np.repeat(' ', len(self.punctuation))))
        
    def remove_punct(self, text):
        table = str.maketrans(self.dict_punct)
        text = text.translate(table)
        return text
    
    def remove_num(self, text):
        num_pattern = r'[\u1369-\u137C]'
        text = re.sub(num_pattern, ' ', text)
        return text
    
    def remove_spaces(self, text):
        try:
            text = re.sub(self.one_space_regex, ' ', text)    
            text = text if text[0] != ' ' else text[1:]
            text = text if text[-1] != ' ' else text[:-1]
            return text
        
        except IndexError as e:
            return ''
    
    def is_am_token(self, token):
        for ch in set(token):
            if ord(ch) < int(self.min_ascii, 16) or ord(ch) > int(self.max_ascii, 16):
                return False
        return True

    def remove_foreign_lang(self, text):
        clean_text = ''
        for token in text.split():
            if self.is_am_token(token):
                clean_text += ' ' + token
        return clean_text[1:]
    
    def clean_text(self, text):
        text = self.remove_punct(text)
        text = self.remove_num(text)
        text = self.remove_spaces(text)
        text = self.remove_foreign_lang(text)
        text = self.remove_spaces(text)
        return text

In [5]:
cleaner = TextCleaner()

In [6]:
cleaned_text = [cleaner.clean_text(text) for text in dirty_corp]

In [7]:
# Initialize the word count to zero
word_count = 0

# Loop over each text in the corpus
for text in cleaned_text:
    # Split the text into words
    words = text.split()
    # Increment the word count by the number of words in this text
    word_count += len(words)

print(f"Total number of words: {word_count}")

Total number of words: 28204888


## Removing stopwords
We will make two versions of the corpus - one without stopwords and the other with them

[Stopwords were taken from here]("https://www.irit.fr/AmharicResources/stop-word-list")

In [8]:
# List of Amharic stopwords were taken from this repository: https://github.com/Amharic-NLP/StopWords
# Load the stopwords from the text file
with open("amharic-stopword.txt", "r", encoding="utf-8") as f:
    stopwords = [line.strip() for line in f]

In [9]:
#stopwords

In [10]:
# Loop over each text in the cleaned corpus
filtered_corpus = []
for text in cleaned_text:
    # Split the text into words
    words = text.split()
    # Remove stop words
    filtered_words = [word for word in words if word not in stopwords]
    # Join the filtered words back into a single text
    filtered_text = ' '.join(filtered_words)
    # Add the filtered text to the filtered corpus
    filtered_corpus.append(filtered_text)

In [11]:
#we will count words after removing stopwords
# Initialize the word count to zero
word_count = 0

# Loop over each text in the corpus   
for text in filtered_corpus:
    # Split the text into words
    words = text.split()
    # Increment the word count by the number of words in this text
    word_count += len(words)

print(f"Total number of words: {word_count}")

Total number of words: 25735662


In [12]:
filtered_corpus[1][1:100]

'ምህርት የገባብ ክፍል ቆሮግቶስ በቁጥር ላይ እኀጺህ እግጺሆኀግልኻ ይህኻ አስጽፍምቅ ቅ ሲል ሞኀ ማለቱ ነው ጥናቄ በቁጥር ላይ ፋትምክሕት የሚለው ምኑገ ነው '

## Character Level Normalization
Amharic has characters wich have the same sound that can be interchangably used.

for example letters 'ሃ','ኅ','ኃ','ሐ','ሓ','ኻ','ሀ' have the same sound so we change them to 'ሀ'

In [13]:
# Character level normalization was taken from here: 
#https://github.com/IsraelAbebe/An-Amharic-News-Text-classification-Dataset/blob/main/Amharic-News-Text-classification-Baseline.ipynb

#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalize_char_level_missmatch(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    #Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል  
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ  
    return rep48

In [14]:
# Tokenize each text in the list
tokenized_texts = [word_tokenize(text) for text in filtered_corpus]
normalized_texts = [[normalize_char_level_missmatch(token) for token in tokenized_text] for tokenized_text in tokenized_texts]

In [15]:
cleaned_corp = [' '.join(tokens) for tokens in normalized_texts]

In [16]:
#will save the file to use it for lemmatization in another place
with open("amharic_normalized_cleaned_text.txt", "w", encoding="utf-8") as file:
    # Write each normalized text on a separate line
    for text in cleaned_corp:
        file.write(text + '^^^^^^')

# Close the text file
file.close()

## Lemmatization

I used the lemmeizer in another place and just get beack the lemmatized text for further preprocessing

## Tokens replacement

In [17]:
#first we import the lemmatized text
with open('./Amharic_normalized_lemmatized_text.txt', 'r', encoding="utf-8") as f:
    cleaned_corp = f.read()
cleaned_corp = cleaned_corp.split('\n')
cleaned_corp = [text for text in cleaned_corp if text != ""] #remove empty texts
#there was some underscore signs after the lemmatizer, I need to remove it 
cleaned_corp = [text.replace("_", "") for text in cleaned_corp] 

In [18]:
len(cleaned_corp)

1320

In [19]:
#Determiners were taken from here: https://en.wiktionary.org/wiki/Category:Amharic_determiners
determiners = ["ሁሉ","ማንኛው","ምንድር", "ምንድን", "ብዙ",  "እቺ", "እች", "እነዚህ", "እነዚያ","ዚህ", "ዚች", "ዚያች","የርሱ", "የርሷ", "የሱ", "የሷ", "የቱ", "የቲቱ", "የትኛው", "የቷ", "የነርሱ", "የነሱ", "የናንተ", "የኔ", "የኛ", "ያቺ", "ያች", "ያንተ", "ያንቺ", "ይህ", "ይህቺ", "ይህች", "ይቺ", "ይች"]
# pronouns were taken from here https://polyglotclub.com/wiki/Language/Amharic/Grammar/Personal-pronouns
pronouns = ["እኔ", "አንተ", "አንቺ", "እሱ", "እሷ", "እኛ", "እናንተ", "እነሱ", "ልጆችዎ"]

In [20]:
#This is a pronouns and determiners replacement
replacements = {
    "pronoun_token": "PRON",
    "determiner_token": "DET"
}
def replace_tokens(text):
    replaced_text = []
    words = text.split()
    for word in words:
        if word in pronouns:
            replaced_text.append(replacements["pronoun_token"])
        elif word in determiners:
            replaced_text.append(replacements["determiner_token"])
        else:
            replaced_text.append(word)
    return replaced_text

replaced_texts = []
for text in cleaned_corp:
    replaced_text = replace_tokens(text)
    replaced_texts.append(replaced_text)

cleaned_corp = [' '.join(tokens) for tokens in replaced_texts]

### Example of the text cleaning

In [21]:
initial_text = dirty_corp[1][1:400]
final_text = cleaned_corp[1][1:400]
print(f'Before: {initial_text}')
print('\n')
print(f'After: {final_text}')

Before: {"page_number": 1, "text": "ትምህርት 5

የገባብ ክፍል:-  (3፳ ቆሮግቶስ 9;35-16)

ጥ2ቄ 3, በቁጥር 15 ላይ «እኀጺህ እግጺሆኀግልኻ ይህኻ አስጽፍምቅ»ቅ ሲል
ሞኀ ማለቱ ነው?

ጥናቄ 2. በቁጥር 35 ላይ ፋትምክሕት» የሚለው ምኑገ ነው?

ጥዩቄ 5, በቁጥር 37 መሠረት ሐዋርደው ወገኻጌልኀ ይሰብክ የነበረው
በፈቃጽ ነበር ወይስ ፻ለፈቃጽሃ

ጥ5ናቄ 4, በቁጥር 38 ላይ ወኀግጌስኻ የመስበክ ጸመወዙ ምገኀጽነው ይላል?

ቁጥር 35;-- ክላይ ክዘረዘረቸው  መብቶች  በአጎገጹም እገኳ
#ልተጠቀመ እግጸሆነ ይመሰክረሪለል።። - በዚህ ገዜ በአእምሮው ዋመጣበት
ምናልባት የቆሮኻቶስ ክርስቲፎኖች «ይህጎገ ሁ


After: ምህርት የገባብ ክፍል ቆሮግቶስ በቁጥር ላይ እኀፂህ እግፂሆኀግልሀ ይህሀ አስፅፍምቅ ቅ ሲል ሞኀ ማለቱ ጥናቄ በቁጥር ላይ ፋትምክህት የሚለው ምኑገ ጥዩቄ በቁጥር መሰረት ሀዋርደው ወገሀጌልኀ ይሰብክ በፈቃፅ ነበር ለፈቃፅሀ በቁጥር ላይ ወኀግጌስሀ የመስበክ ፀመወዙ ምገኀፅነው ቁጥር ክላይ ክዘረዘረቸው መብቶች በአጎገፁም እገኳ ልተጠቀመ እግፀሆነ ይመሰክረሪለል ገዜ በአእምሮው ዋመጣበት የቆሮሀቶስ ክርስቲፎኖች ይህጎገ አሁሀ የሚናገረው ፀመወዝ ፈልጐ ይሉኝ ይሆናለ የሚለው ግምት ነበር ይህም እኀፃለሆነ በክባፅ አነጋገር ዌሪጋግጥላቸዋል ማም ትምክህቴሀ ክሀቱ ከሚፀርግብኝ ሞት ይሻለናልና ትምክህቱ ወግጌለኘሀ በነአ መስበኩ ግሀ አሁሀ ፃሳ


In [22]:
with open("amharic_lemmatized_cleaned_text.txt", "w", encoding="utf-8") as file:   
    for text in cleaned_corp:
        file.write(text + '\n')
file.close()